![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Revisiting Machine Learning Case Study

- In this lab, you will use `learningSet.csv` file which you already have cloned in today's activities. # HEALTHCARE FOR ALL STUDY CASE

In [212]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pyplot
import seaborn as sns

In [213]:
# Import data
data=pd.read_csv('C:/Users/aguva/Ironhack/Unit7/7.1/learningSet.csv')

In [214]:
# Observamos la dimensiónd del Dataset
data.shape

(95412, 481)

- Check for null values in all the columns

In [215]:
# Observamos el % de nulos en cada columna

nulls_percent_df=pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['columns_name', 'nulls_percentage']
nulls_percent_df

,columns_name,nulls_percentage
0,ODATEDW,0.000000
1,OSOURCE,0.000000
2,TCODE,0.000000
3,STATE,0.000000
4,ZIP,0.000000
...,...,...
476,MDMAUD_R,0.000000
477,MDMAUD_F,0.000000
478,MDMAUD_A,0.000000
479,CLUSTER2,0.001383


In [216]:
nulls_percent_df.sort_values(by=['nulls_percentage'], ascending=False).head(5)

,columns_name,nulls_percentage
414,RDATE_5,0.999906
436,RAMNT_5,0.999906
412,RDATE_3,0.997464
434,RAMNT_3,0.997464
413,RDATE_4,0.997055


- Exclude the following variables by looking at the definitions. Create a new empty list called `drop_list`. We will append this list and then drop all the columns in this list later:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP CODE` - we are including state already -No existe elimino `ZIP`

In [217]:
# Columnas que quiero remover
drop_columns_list = ['OSOURCE', 'ZIP']

# Verifica si están en el DataFrame
existing_cols = data.columns.tolist()

for col in drop_columns_list:
    if col in existing_cols:
        existing_cols.remove(col)

# Eliminar las columnas de la lista en el DataFrame
data = data.drop(drop_columns_list, axis=1)

# Limpiar la lista drop_columns_list
drop_columns_list.clear()

# Verificar la forma del DataFrame después de eliminar las columnas
print(data.shape)


(95412, 479)


- Identify columns that over 85% missing values

In [218]:
# Create subset with columns with => 0.25 of null values
df_85=nulls_percent_df[nulls_percent_df['nulls_percentage']>0.85]

df_85

,columns_name,nulls_percentage
23,NUMCHLD,0.870184
412,RDATE_3,0.997464
413,RDATE_4,0.997055
414,RDATE_5,0.999906
415,RDATE_6,0.991867
416,RDATE_7,0.906773
419,RDATE_10,0.890360
422,RDATE_13,0.871609
424,RDATE_15,0.923888
426,RDATE_17,0.901469


- Remove those columns from the dataframe

In [219]:
# Create subset with columns with => 0.85 of null values
drop_columns=nulls_percent_df[nulls_percent_df['nulls_percentage']>0.85]

# Obtener una lista de nombres de columna desde el DataFrame drop_columns
drop_columns_list = drop_columns['columns_name'].tolist()

# Verifica si están en el DataFrame
existing_cols = data.columns.tolist()

for col in drop_columns_list:
    if col in existing_cols:
        existing_cols.remove(col)

# Eliminar las columnas de la lista en el DataFrame
data = data.drop(drop_columns_list, axis=1)

# Limpiar la lista drop_columns_list
drop_columns_list.clear()

# Verificar la forma del DataFrame después de eliminar las columnas
print(data.shape)

(95412, 454)


- Reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```

In [220]:
data_cat = data.select_dtypes(object)

data_cat.shape

(95412, 72)

In [221]:
data_cat.GENDER.value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [222]:
data_cat.GENDER.isnull().sum()

0

In [223]:
data_cat.GENDER.fillna('F', inplace=True)

In [224]:
# Definimos una función llamada clean_gender
def clean_gender(x):
     # Verificar si x no está en la lista ['F', 'M']
    if x not in ['F', 'M']:
        # Si no está en la lista, asignar 'other' y devolverlo
        return 'other'
    else:
        # Si está en la lista, devolver x sin cambios
        return x

In [225]:
data_cat.GENDER= list(map(clean_gender, data_cat['GENDER']))
data_cat.GENDER.value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64

![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Feature engineering

- In this lab, you will use `learningSet.csv` file which you have already cloned in the previous activities. 

### Instructions


Here we will work on cleaning some of the other columns in the dataset using the techniques that we used before in the lessons.

- Check for null values in the numerical columns.

In [226]:
data_num = data.select_dtypes(np.number)

data_num.shape

(95412, 382)

In [227]:
# Observamos el % de nulos en cada columna en el DataFrame numerico

nulls_percent_df=pd.DataFrame(data_num.isna().sum()/len(data_num)).reset_index()
nulls_percent_df.columns = ['columns_name', 'nulls_percentage']
nulls_percent_df

,columns_name,nulls_percentage
0,ODATEDW,0.000000
1,TCODE,0.000000
2,DOB,0.000000
3,AGE,0.248030
4,INCOME,0.223096
...,...,...
377,TARGET_B,0.000000
378,TARGET_D,0.000000
379,HPHONE_D,0.000000
380,RFA_2F,0.000000


In [228]:
nulls_percent_df.sort_values(by=['nulls_percentage'], ascending=False).head(20)

,columns_name,nulls_percentage
345,RDATE_11,0.845512
355,RAMNT_11,0.845512
360,RAMNT_19,0.833595
350,RDATE_19,0.833595
354,RAMNT_9,0.824613
344,RDATE_9,0.824613
362,RAMNT_24,0.814090
352,RDATE_24,0.814090
349,RDATE_18,0.792710
359,RAMNT_18,0.792710


- Use appropriate methods to clean the columns `GEOCODE2`, `WEALTH1`, `ADI`, `DMA`,and `MSA`.

In [229]:
data_num.MSA.value_counts()

0.0       21333
4480.0     4606
1600.0     4059
2160.0     2586
520.0      1685
          ...  
9140.0        1
3200.0        1
9280.0        1
743.0         1
8480.0        1
Name: MSA, Length: 298, dtype: int64

In [230]:
# No esta 'GEOCODE2'

drop_columns = ['WEALTH1', 'ADI', 'DMA', 'MSA']

# Verifica si están en el DataFrame
existing_cols = data_num.columns.tolist()

for col in drop_columns:
    if col in existing_cols:
        existing_cols.remove(col)

# Eliminar las columnas de la lista en el DataFrame
data_num = data_num.drop(drop_columns, axis=1)

# Limpiar la lista drop_columns
drop_columns.clear()

# Verificar la forma del DataFrame después de eliminar las columnas
print(data_num.shape)


(95412, 378)


- Use appropriate EDA technique where ever necessary.

* Numerical

In [231]:
data_num = data_num.apply(lambda x: x.replace(" ", np.nan))

In [232]:
# DataFrame Numerico
nulls_percent_df_num=pd.DataFrame(data_num.isna().sum()/len(data_num)).reset_index()
nulls_percent_df_num.columns = ['columns_name', 'nulls_percentage']
nulls_percent_df_num.shape

(378, 2)

In [233]:
# Create subset with columns with => 0.25 of null values
drop_columns_num=nulls_percent_df_num[nulls_percent_df_num['nulls_percentage']>0.25]
drop_columns_num.shape

(46, 2)

In [234]:
# Obtener una lista de nombres de columna desde el DataFrame drop_columns
drop_columns_list = drop_columns_num['columns_name'].tolist()

# Verificar si están en el DataFrame
existing_cols = data_num.columns.tolist()

for col in drop_columns_list:
    if col in existing_cols:
        existing_cols.remove(col)

# Eliminar las columnas de la lista en el DataFrame
data_num = data_num.drop(drop_columns_list, axis=1)

# Limpiar la lista drop_columns_list
drop_columns_list.clear()

# Verificar la forma del DataFrame después de eliminar las columnas
print(data_num.shape)


(95412, 332)


* Categorical

In [235]:
data_cat = data_cat.apply(lambda x: x.replace(" ", np.nan))

In [236]:
# DataFrame Categorico
nulls_percent_df_cat=pd.DataFrame(data_cat.isna().sum()/len(data_cat)).reset_index()
nulls_percent_df_cat.columns = ['columns_name', 'nulls_percentage']
nulls_percent_df_cat

,columns_name,nulls_percentage
0,STATE,0.000000
1,MAILCODE,0.985337
2,PVASTATE,0.984719
3,NOEXCH,0.000073
4,RECINHSE,0.929747
...,...,...
67,RFA_2A,0.000000
68,MDMAUD_R,0.000000
69,MDMAUD_F,0.000000
70,MDMAUD_A,0.000000


In [237]:
# Pautas de la actividad
data_cat['SOLIH'] = data_cat['SOLIH'].fillna("13")
data['VETERANS'] = data['VETERANS'].fillna("N")

In [238]:
# Agruparemos los estados que tienen =< 2500 donadores

df= pd.DataFrame(data_cat['STATE'].value_counts()).reset_index()

df.columns= ['state', 'count']

other= list(df[df['count'] <= 2500]['state'])
other

['TN',
 'AZ',
 'OR',
 'MN',
 'CO',
 'SC',
 'AL',
 'KY',
 'OK',
 'LA',
 'KS',
 'IA',
 'AR',
 'MS',
 'NV',
 'NM',
 'NE',
 'UT',
 'ID',
 'MT',
 'HI',
 'SD',
 'WY',
 'AK',
 'ND',
 'AP',
 'NY',
 'VA',
 'OH',
 'PA',
 'MD',
 'NJ',
 'MA',
 'CT',
 'AA',
 'AE',
 'ME',
 'NH',
 'VT',
 'RI',
 'VI',
 'WV',
 'GU',
 'DE',
 'DC',
 'AS']

In [239]:
# Mapeo para transformar los estados con donadores menores a 2500 agruparlos en Other

def other_state(x):
    if x in other:
        return 'Other'
    else:
        return x


In [240]:
data_cat['STATE'] = list(map(other_state, data_cat['STATE']))
data_cat.STATE.value_counts()

Other    30457
CA       17343
FL        8376
TX        7535
IL        6420
MI        5654
NC        4160
WA        3577
GA        3403
IN        2980
WI        2795
MO        2712
Name: STATE, dtype: int64

In [241]:
data_cat.columns

Index(['STATE', 'MAILCODE', 'PVASTATE', 'NOEXCH', 'RECINHSE', 'RECP3',
       'RECPGVG', 'RECSWEEP', 'MDMAUD', 'DOMAIN', 'CLUSTER', 'AGEFLAG',
       'HOMEOWNR', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'GENDER',
       'DATASRCE', 'SOLP3', 'SOLIH', 'MAJOR', 'GEOCODE', 'COLLECT1',
       'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
       'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER',
       'KIDSTUFF', 'CARDS', 'PLATES', 'LIFESRC', 'PEPSTRFL', 'RFA_2', 'RFA_3',
       'RFA_4', 'RFA_5', 'RFA_6', 'RFA_7', 'RFA_8', 'RFA_9', 'RFA_10',
       'RFA_11', 'RFA_12', 'RFA_13', 'RFA_14', 'RFA_15', 'RFA_16', 'RFA_17',
       'RFA_18', 'RFA_19', 'RFA_20', 'RFA_21', 'RFA_22', 'RFA_23', 'RFA_24',
       'RFA_2R', 'RFA_2A', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A', 'GEOCODE2'],
      dtype='object')

In [208]:
# Create subset with columns with => 0.25 of null values
drop_columns_cat=nulls_percent_df_cat[nulls_percent_df_cat['nulls_percentage']>0.25]
drop_columns_cat

,columns_name,nulls_percentage
1,MAILCODE,0.985337
2,PVASTATE,0.984719
4,RECINHSE,0.929747
5,RECP3,0.978860
6,RECPGVG,0.998805
7,RECSWEEP,0.983052
11,AGEFLAG,0.309689
13,CHILD03,0.987989
14,CHILD07,0.983587
15,CHILD12,0.981019


In [209]:
# Obtener una lista de nombres de columna desde el DataFrame drop_columns
drop_columns_list = drop_columns_cat['columns_name'].tolist()

# Verificar si están en el DataFrame
existing_cols = drop_columns_cat.columns.tolist()

for col in drop_columns_list:
    if col in existing_cols:
        existing_cols.remove(col)

# Eliminar las columnas de la lista en el DataFrame
data_cat = data_cat.drop(drop_columns_list, axis=1)

# Limpiar la lista drop_columns_list
drop_columns_list.clear()

# Verificar la forma del DataFrame después de eliminar las columnas
print(data_cat.shape)

(95412, 26)


In [210]:
data_cat

,STATE,NOEXCH,MDMAUD,DOMAIN,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2,RFA_3,...,RFA_12,RFA_14,RFA_16,RFA_18,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,IL,0,XXXX,T2,36,NaN,F,NaN,L4E,S4E,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,CA,0,XXXX,S1,14,H,M,3,L2G,A2G,...,A1E,NaN,L1E,NaN,L,G,X,X,X,A
2,NC,0,XXXX,R2,43,U,M,3,L4E,S4E,...,S4F,S4F,S4F,S4D,L,E,X,X,X,C
3,CA,0,XXXX,R2,44,U,F,3,L4E,S4E,...,S4E,S4E,S4E,S2D,L,E,X,X,X,C
4,FL,0,XXXX,S2,16,H,F,3,L2F,A2F,...,A1E,L3D,L3D,A2D,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,Other,0,XXXX,C2,27,NaN,M,NaN,L1G,F1G,...,NaN,NaN,NaN,NaN,L,G,X,X,X,C
95408,TX,0,XXXX,C1,24,H,M,3,L1F,NaN,...,NaN,NaN,NaN,NaN,L,F,X,X,X,A
95409,MI,0,XXXX,C3,30,NaN,M,NaN,L3E,S4E,...,N3E,N3E,F1D,F1D,L,E,X,X,X,B
95410,CA,0,XXXX,C1,24,H,F,2,L4F,S4F,...,S4F,S4F,S3F,S2F,L,F,X,X,X,A
